In [ ]:
#pip install unidecode

In [ ]:
from sklearn.feature_selection import SelectKBest, f_regression, f_classif
import pandas as pd
import numpy as np
import random
import warnings
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.feature_selection import RFE
from sklearn.decomposition import PCA
from sklearn.metrics import roc_auc_score, mean_squared_error, r2_score
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import cross_val_score
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, roc_auc_score, mean_squared_error, r2_score, confusion_matrix
from sklearn.utils import check_array
from unidecode import unidecode
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn.linear_model import ElasticNet
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_percentage_error
from sklearn.linear_model import LogisticRegression
import xgboost as xgb
from imblearn.over_sampling import RandomOverSampler
from sklearn.model_selection import learning_curve



In [ ]:
def balancear_datos(X, Y):
    # Instancia el RandomOverSampler
    ros = RandomOverSampler(random_state=13)

    # Aplica la técnica de sobremuestreo para balancear las clases
    X_resampled, Y_resampled = ros.fit_resample(X, Y)

    return X_resampled, Y_resampled
print('balancear_datos Loaded')

balancear_datos Loaded


In [ ]:
# SelectKBest : Selecciona las variables Relevantes Utilizando SelectKBest

def Selectkbest_features(X, y, k,tipo_modelo=f_regression):
    selector = SelectKBest(score_func=f_regression, k=k)  # Utilizamos f_regression como función de puntuación
    X_new = selector.fit_transform(X, y)  # Seleccionamos las k mejores características

    mask = selector.get_support()  # Máscara booleana de las características seleccionadas
    selected_features = [feature for feature, mask_value in zip(X.columns, mask) if mask_value]
    selected_scores = selector.scores_[selector.get_support()]

    # Crear un DataFrame con las variables seleccionadas y sus puntuaciones
    selected_data = pd.DataFrame({'Variable': selected_features, 'Puntuacion': selected_scores})

#f_regression; f_classif
    return selected_data
print('Selectkbest_features Loaded')

Selectkbest_features Loaded


In [ ]:
# RFE: Fucnion que obtiene las variables utilizanfo el modelo de Recursive Feature Elimination.

def rfe_features(estimator, X, y, n_features_to_select=None):

    warnings.filterwarnings("ignore", category=DeprecationWarning)
    selector = RFE(estimator, n_features_to_select=n_features_to_select)
    selector.fit(X, y)

    # Obtener las variables seleccionadas
    selected_features = X.columns[selector.support_].tolist()


    # Obtener los puntajes de las características seleccionadas
    feature_scores = selector.estimator_.coef_

    # Crear el DataFrame de resultado
    df_result = pd.DataFrame({'Variable': selected_features, 'Score': feature_scores})
    warnings.filterwarnings("default", category=DeprecationWarning)
    return df_result
print('rfe_features Loaded')

rfe_features Loaded


In [ ]:
 # Integración de Variables

def integrate_features(Selectkbest_List,Boruta_List,Rfe_List):

    integrated_features = list(set(Selectkbest_List + Boruta_List + Rfe_List))

    return integrated_features
print('Integración de Variables Loaded')

Integración de Variables Loaded


In [ ]:
# Funcion de evaluacion de los modelos

def evaluate_results(y_true, y_pred):
    # Curva ROC
    roc_auc = roc_auc_score(y_true, y_pred)

    # Error cuadrático medio (MSE)
    mse = mean_squared_error(y_true, y_pred)

    # Raíz del error cuadrático medio (RMSE)
    rmse = np.sqrt(mse)

    # Coeficiente de determinación (R2)
    r2 = r2_score(y_true, y_pred)

    # Error porcentual absoluto medio (MAPE)
    mape = np.mean(np.abs((y_true - y_pred) / y_true)) * 100

    return roc_auc, mse, rmse, r2, mape
print('evaluacion de los modelos Loaded')

In [ ]:
# Generacion de Componentes Principales

def generar_componentes_principales_X(dataframe, n_componentes):
    # Inicializar el objeto PCA con el número de componentes deseados
    pca = PCA(n_components=n_componentes)

    # Realizar el PCA y transformar los datos
    componentes_principales = pca.fit_transform(dataframe)

    # Crear un DataFrame con los componentes principales
    df_componentes = pd.DataFrame(data=componentes_principales, columns=[f'Componente_{i+1}' for i in range(n_componentes)])

    return df_componentes
print('Pca_components Loaded')

Pca_components Loaded


In [ ]:
#Funcion que encuentra las variables utilziando tensorflow

def find_important_variables(X, y, model_type='random_forest', n_estimators=100):
    # Configurar TensorFlow para utilizar la GPU
    physical_devices = tf.config.list_physical_devices('GPU')
    tf.config.experimental.set_memory_growth(physical_devices[0], True)

    if model_type == 'random_forest':
        # Crear el modelo Random Forest
        model = RandomForestRegressor(n_estimators=n_estimators)

    elif model_type == 'xgboost':
        # Crear el modelo XGBoost
        model = XGBRegressor(n_estimators=n_estimators)

    else:
        raise ValueError('Model type not supported.')

    # Convertir los datos a tensores de TensorFlow
    X_tensor = tf.convert_to_tensor(X.values, dtype=tf.float32)
    y_tensor = tf.convert_to_tensor(y.values, dtype=tf.float32)

    # Entrenar el modelo
    with tf.device('/GPU:0'):
        model.fit(X_tensor, y_tensor)

    # Obtener la importancia de las variables
    if model_type == 'random_forest':
        feature_importances = model.feature_importances_
    elif model_type == 'xgboost':
        feature_importances = model.feature_importances_
    else:
        feature_importances = None

    if feature_importances is not None:
        # Obtener los nombres de las variables
        variable_names = X.columns

        # Crear un DataFrame con las variables y su importancia
        importance_df = pd.DataFrame({'Variable': variable_names, 'Importance': feature_importances})

        # Ordenar las variables por su importancia en orden descendente
        importance_df = importance_df.sort_values('Importance', ascending=False).reset_index(drop=True)

        return importance_df
    else:
        raise ValueError('Feature importances not available.')
print('Improtancia con XGBoost y Random forest Loaded')

In [ ]:
#Modelo para encontrar variables importantes usando XGBOOST y RANDOM FOREST
def find_important_variables_Clasificacion(X, y, model_type='xgboost', n_estimators=100):
    if model_type == 'xgboost':
        # Crear el modelo XGBoost clasificador
        model = xgb.XGBClassifier(n_estimators=n_estimators)

    elif model_type == 'random_forest':
        # Crear el modelo Random Forest clasificador
        model = RandomForestClassifier(n_estimators=n_estimators, random_state=42)

    else:
        raise ValueError('Model type not supported. Use "xgboost" or "random_forest".')


    # Dividir los datos en conjuntos de entrenamiento y prueba
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Entrenar el modelo
    model.fit(X_train, y_train)

    # Obtener la importancia de las variables
    feature_importances = model.feature_importances_

    # Obtener los nombres de las variables
    variable_names = X.columns

    # Crear un DataFrame con las variables y su importancia
    importance_df = pd.DataFrame({'Variable': variable_names, 'Importance': feature_importances})

    # Ordenar las variables por su importancia en orden descendente
    importance_df = importance_df.sort_values('Importance', ascending=False).reset_index(drop=True)

    return importance_df

In [ ]:
def xgboost_model_evaluation(X_train, X_test, y_train, y_test,param_grid):

    # Crear el modelo XGBoost
    xgb_model = XGBRegressor()

    # Realizar Grid Search con validación cruzada
    grid_search = GridSearchCV(estimator=xgb_model, param_grid=param_grid, scoring='neg_mean_squared_error', cv=5)
    grid_search.fit(X_train, y_train)

    # Obtener los mejores hiperparámetros
    best_params = grid_search.best_params_

    # Crear el modelo XGBoost con los mejores hiperparámetros
    xgb_best_model = XGBRegressor(**best_params)

    # Ajustar el modelo con los mejores hiperparámetros a los datos de entrenamiento
    xgb_best_model.fit(X_train, y_train)

    # Realizar la predicción en el grupo de prueba
    y_pred = xgb_best_model.predict(X_test)

    # Calcular el error cuadrático medio (MSE) del modelo en el grupo de prueba
    mse = mean_squared_error(y_test, y_pred)

    # Calcular el coeficiente de determinación (R2) del modelo en el grupo de prueba
    r2 = r2_score(y_test, y_pred)

    # Calcular el error porcentual absoluto medio (MAPE) del modelo en el grupo de prueba
    mape = mean_absolute_percentage_error(y_test, y_pred)

    return xgb_best_model,y_pred, best_params, mse, r2, mape

In [ ]:
def random_forest_regression(X_train, X_test, y_train, y_test, param_grid):
    # Paso 1: Definir el modelo de Random Forest para regresión
    rf_model = RandomForestRegressor()

    # Paso 2: Realizar la búsqueda de hiperparámetros utilizando GridSearchCV
    grid_search = GridSearchCV(estimator=rf_model, param_grid=param_grid, cv=5)
    grid_search.fit(X_train, y_train)

    # Paso 3: Obtener el mejor modelo y los mejores parámetros
    xgb_best_model = grid_search.best_estimator_
    best_params = grid_search.best_params_

    # Paso 4: Realizar predicciones y calcular las métricas de evaluación
    y_pred = xgb_best_model.predict(X_test)

    return xgb_best_model, y_pred, best_params

In [ ]:
def random_forest_model_evaluation(X_train, X_test, y_train, y_test,param_grid):
    """
    Genera la predicción y evaluación de un modelo RandomForestRegressor utilizando Grid Search para encontrar los mejores hiperparámetros.

    Parámetros:
    - X_train: matriz de características para el grupo de entrenamiento.
    - X_test: matriz de características para el grupo de prueba.
    - y_train: vector de etiquetas para el grupo de entrenamiento.
    - y_test: vector de etiquetas para el grupo de prueba.

    Retorna:
    - y_pred: predicciones del modelo en el grupo de prueba.
    - best_params: mejores hiperparámetros encontrados mediante Grid Search.
    - accuracy: exactitud (accuracy) del modelo en el grupo de prueba.
    """

    # Crear el modelo RandomForestRegressor
    rf_model = RandomForestRegressor()

    # Realizar Grid Search con validación cruzada
    grid_search = GridSearchCV(estimator=rf_model, param_grid=param_grid, scoring='neg_mean_squared_error', cv=5)
    grid_search.fit(X_train, y_train)

    # Obtener los mejores hiperparámetros
    best_params = grid_search.best_params_

    # Crear el modelo RandomForestRegressor con los mejores hiperparámetros
    rf_best_model = RandomForestRegressor(**best_params)

    # Ajustar el modelo con los mejores hiperparámetros a los datos de entrenamiento
    rf_best_model.fit(X_train, y_train)

    # Realizar la predicción en el grupo de prueba
    y_pred = rf_best_model.predict(X_test)

    # Calcular la exactitud del modelo en el grupo de prueba
    #accuracy = accuracy_score(y_test, y_pred)

    return y_pred, best_params
print('Random Forest Loaded')

Random Forest Loaded


In [ ]:
def obtener_resultados_modelo(modelo, X, y):
    # Obtener las etiquetas predichas por el modelo
    y_pred = modelo.predict(X)

    # Convertir 'y_pred' de ndarray a DataFrame
    y_pred_df = pd.DataFrame(y_pred, columns=['y_pred'])

    # Reiniciar los índices para asegurar la alineación correcta
    y.reset_index(drop=True, inplace=True)
    y_pred_df.reset_index(drop=True, inplace=True)

    # Concatenar 'y' y 'y_pred' a lo largo de las columnas
    result_df = pd.concat([y, y_pred_df], axis=1)
    r2 = r2_score(result_df['Crecimiento'], result_df['y_pred'])
    mse = mean_squared_error(result_df['Crecimiento'], result_df['y_pred'])
    mape = mape = np.mean(np.abs((result_df['Crecimiento']- result_df['y_pred']) / result_df['Crecimiento'])) * 100

    # Calcular la diferencia entre 'y' y 'y_pred'
    result_df['diferencia'] = result_df['Crecimiento'] - result_df['y_pred']
    result_df['diferencia2'] = (result_df['Crecimiento'] - result_df['y_pred'])/result_df['Crecimiento']

    return result_df,r2,mse,mape

In [ ]:
##Función que elimina las tildes y diacríticos de un texto.
def quitar_tildes(texto):
    texto_sin_tildes = unidecode(texto)
    return texto_sin_tildes
print('quitar_tildes Loaded')

quitar_tildes Loaded


In [ ]:
##Función completa datos faltantes de serie de timepo
def completar_datos_serie_tiempo(df,inicio,fin):

    # Convert 'Year' to datetime type with date set to December 31st
    df['Year'] = pd.to_datetime(df['Year'].astype(str) + '-12-31')

    # Set 'Year' and 'Category' as the index
    df.set_index(['Year', 'Category'], inplace=True)

    # Create a new DataFrame with all years and categories from the first year to the last year in the available data
    start_year = inicio
    end_year = fin
    all_years = pd.DataFrame(index=pd.MultiIndex.from_product([[str(y) + '-12-31' for y in range(start_year, end_year+1)], df.index.get_level_values('Category').unique()], names=['Year', 'Category']))


    all_years['join']=all_years.index.get_level_values('Year').astype(str) + '-' + all_years.index.get_level_values('Category').astype(str)
    df['join']=df.index.get_level_values('Year').astype(str) + '-' + df.index.get_level_values('Category').astype(str)


    # Join the existing DataFrame with all_years
    all_years.reset_index(inplace=True)
    all_years['Year'] = pd.to_datetime(all_years['Year'])
    df_complete = pd.merge(all_years, df,on='join',how='left')
    #df_complete_s=df_complete
    df_complete.set_index(['Year', 'Category'], inplace=True)
    df_complete = df_complete.drop('join', axis=1)


    # Obtener la lista de columnas que necesitan ser completadas
    columns_to_interpolate = df_complete.columns.tolist()


    for column in columns_to_interpolate:
        df_complete[column] = df_complete.groupby('Category',group_keys=False)[column].apply(lambda x: x.interpolate(limit_direction='both'))

    df_complete.reset_index(inplace=True)
    df_complete=df_complete.fillna(0)
    df_complete['Year'] = df_complete['Year'].dt.year


    return df_complete
print('completar_datos_serie_tiempo Loaded')

completar_datos_serie_tiempo Loaded


In [ ]:
#elimina las filas con NAN
def eliminar_filas_nan(dataframe, columna):
    dataframe_sin_nan = dataframe.dropna(subset=[columna], inplace=False)
    return dataframe_sin_nan
print('eliminar_filas_nan Loaded')

eliminar_filas_nan Loaded


In [ ]:
#si encuentra numeros menores los convierte en Nan
def cambiar_numeros_menores(dataframe, valor, columna_excluida):
    for columna in dataframe.columns:
        if columna != columna_excluida:
            dataframe[columna] = np.where(dataframe[columna] < valor, np.nan, dataframe[columna])
    return dataframe
print('cambiar_numeros_menores Loaded')

cambiar_numeros_menores Loaded


In [ ]:
#Funcion que Normaliza variables numericas

def normalize_variables(dataset):
    scaler = MinMaxScaler()
    normalized_dataset = scaler.fit_transform(dataset)
    normalized_dataset = pd.DataFrame(normalized_dataset, columns=dataset.columns)
    return normalized_dataset
print('normalize_variables Loaded')

normalize_variables Loaded


In [ ]:
def normalize_variables_limpieza(dataset):
    # Eliminar valores infinitos o muy grandes
    dataset = dataset.replace([np.inf, -np.inf], np.nan)
    dataset = dataset.fillna(0)  # Reemplazar valores NaN por cero o cualquier otro valor apropiado

    scaler = MinMaxScaler()
    normalized_dataset = scaler.fit_transform(dataset)
    normalized_dataset = pd.DataFrame(normalized_dataset, columns=dataset.columns)
    return normalized_dataset

In [ ]:
#Calcula el crecimiento respecto a un año especifico
def calcular_crecimiento_ingresos(data, years_offset,Columna,ingresos,Yearp):
    data_sorted = data.sort_values(by=[Yearp])
    data_pivot = data_sorted.pivot(index=Columna, columns=Yearp, values=ingresos)

    # Obtener todas las columnas de años en el DataFrame
    years = data_pivot.columns.tolist()

    crecimiento_data = []

    if years_offset < 0:
        years_offset = abs(years_offset)
        for i in range(years_offset, len(years)):
            current_year = years[i]
            past_year = years[i - years_offset]

            crecimiento = (data_pivot[current_year] - data_pivot[past_year]) / data_pivot[past_year]

            crecimiento_data.append(pd.DataFrame({Columna: data_pivot.index, Yearp: current_year, 'Crecimiento': crecimiento}))
    else:
        for i in range(len(years) - years_offset):
            current_year = years[i]
            future_year = years[i + years_offset]

            crecimiento = (data_pivot[future_year] - data_pivot[current_year]) / data_pivot[current_year]

            crecimiento_data.append(pd.DataFrame({Columna: data_pivot.index, Yearp: current_year, 'Crecimiento': crecimiento}))

    result = pd.concat(crecimiento_data, ignore_index=True)

    return result
print('calcular_crecimiento_ingresos Loaded')

calcular_crecimiento_ingresos Loaded


In [ ]:
# Genera grupos de variables que comparten alta correlacion
def obtener_variables_correlacionadas(data, umbral):
    # Calcular la matriz de correlación
    matriz_correlacion = data.corr().abs()

    # Crear un diccionario para mapear las variables con sus grupos
    grupos = {}
    num_grupo = 1

    # Recorrer las variables y asignarles el número de grupo
    for variable in matriz_correlacion.columns:
        if variable not in grupos:
            variables_correlacionadas = matriz_correlacion.index[matriz_correlacion[variable] >= umbral]
            for var in variables_correlacionadas:
                grupos[var] = num_grupo
            num_grupo += 1

    # Crear un DataFrame con las variables agrupadas
    df_variables_correlacionadas = pd.DataFrame({'Variable': list(grupos.keys()), 'Grupo': list(grupos.values())})

    return df_variables_correlacionadas
print('obtener_variables_correlacionadas Loaded')

obtener_variables_correlacionadas Loaded


In [ ]:

def generar_componentes_principales(df_datos, df_grupos):
    grupos = df_grupos['Grupo'].unique()  # Obtener la lista de grupos únicos
    componentes_p = pd.DataFrame()
    for grupo in grupos:
        # Filtrar las variables correspondientes al grupo actual
        variables_grupo = df_grupos[df_grupos['Grupo'] == grupo]['Variable'].tolist()

        # Filtrar las columnas del DataFrame de datos según las variables del grupo actual
        datos_grupo = df_datos[variables_grupo]

        # Aplicar PCA al DataFrame del grupo actual
        pca = PCA(n_components=1)
        componente_principal = pca.fit_transform(datos_grupo)

        # Asignar nombre al componente principal
        nombre_componente = "Componente_Principal_" + str(grupo)
        df_temp = pd.DataFrame(componente_principal, columns=[nombre_componente])
        componentes_p = pd.concat([componentes_p, df_temp], axis=1)
        # Agregar el componente principal al DataFrame de datos

    return componentes_p
print('generar_componentes_principales Loaded')

generar_componentes_principales Loaded


In [ ]:
# RFE: Fucnion que obtiene las variables utilizanfo el modelo de Recursive Feature Elimination.

def rfe_with_elasticnet(X, y, n_features_to_select):
    # Crear el estimador ElasticNet
    estimator = ElasticNet()

    # Crear el objeto RFE
    rfe = RFE(estimator=estimator, n_features_to_select=n_features_to_select)

    # Realizar la selección de características
    rfe.fit(X, y)

    # Obtener los nombres de las características
    feature_names = X.columns

    # Obtener los coeficientes de importancia de las características
    feature_importances = rfe.estimator_.coef_

    # Crear un DataFrame con las características y sus coeficientes de importancia
    result_df = pd.DataFrame({'Feature': feature_names, 'Importance': feature_importances})

    return result_df
print('rfe_with_elasticnet Loaded')

In [ ]:
# Funcion para calcular RFE En classificación

def select_features_with_rfe_Clasisification(X, y, num_features):
    # Instanciar el modelo de clasificación (por ejemplo, Regresión Logística)
    model = LogisticRegression(max_iter=1000)

    # Crear el objeto RFE y especificar el modelo y el número de características deseado
    rfe = RFE(model, n_features_to_select=num_features)

    # Ajustar RFE a los datos
    rfe.fit(X, y)

    # Obtener las características seleccionadas con sus coeficientes de importancia
    selected_features = X.columns[rfe.support_]
    coef_importance = rfe.estimator_.coef_[0]

    # Crear un DataFrame con las características seleccionadas y sus coeficientes de importancia
    result_df = pd.DataFrame({'Variable': selected_features, 'Importance': coef_importance})

    return result_df

In [ ]:
def calcular_crecimiento_promedio_por_categoria_Predial(df, Year_inicio_periodo1, Year_fin_periodo1, Year_inicio_periodo2, Year_fin_periodo2):
    # Filtrar el DataFrame por los años de cada periodo
    periodo1 = df[(df['Año'] >= Year_inicio_periodo1) & (df['Año'] <= Year_fin_periodo1)]
    periodo2 = df[(df['Año'] >= Year_inicio_periodo2) & (df['Año'] <= Year_fin_periodo2)]

    # Calcular el promedio de valores para cada categoría en cada periodo
    promedio_periodo1 = periodo1.groupby('CODIGO_DANE')['Predial'].mean()
    promedio_periodo2 = periodo2.groupby('CODIGO_DANE')['Predial'].mean()

    # Calcular el crecimiento porcentual promedio para cada categoría
    crecimiento_promedio = ((promedio_periodo2 - promedio_periodo1) / promedio_periodo1) * 100

    return crecimiento_promedio
print('calcular_crecimiento_promedio_por_categoria_Predial Loaded')

In [ ]:
def categorizar_por_boxplot(df,columna_valores):
    # Calcular el primer, segundo y tercer cuartil
    primer_cuartil = columna_valores.quantile(0.25)
    segundo_cuartil = columna_valores.quantile(0.5)
    tercer_cuartil = columna_valores.quantile(0.75)

    # Calcular los límites del bigote inferior y superior
    iqr = tercer_cuartil - primer_cuartil
    limite_inferior = primer_cuartil - 1.5 * iqr
    limite_superior = tercer_cuartil + 1.5 * iqr

    # Categorizar los valores según los análisis de boxplot
    categorias = []
    for valor in columna_valores:
        if valor < limite_inferior:
            categorias.append('Muy baja')
        elif valor < primer_cuartil:
            categorias.append('Baja')
        elif valor < tercer_cuartil:
            categorias.append('Media')
        elif valor <= limite_superior:
            categorias.append('Alta')
        else:
            categorias.append('Muy alta')
    df['recomendacion'] = categorias
    return categorias
print('categorizar_por_boxplot Loaded')

In [ ]:
def calcular_crecimiento_promedio_por_categoria(df, columnas_procesar, Year_inicio_periodo1, Year_fin_periodo1, Year_inicio_periodo2, Year_fin_periodo2):
    # Filtrar el DataFrame por los años de cada periodo
    periodo1 = df[(df['Year'] >= Year_inicio_periodo1) & (df['Year'] <= Year_fin_periodo1)]
    periodo2 = df[(df['Year'] >= Year_inicio_periodo2) & (df['Year'] <= Year_fin_periodo2)]

    # Crear un DataFrame vacío para almacenar los resultados
    resultados = pd.DataFrame()

    # Iterar sobre las columnas y calcular el crecimiento por categoría
    for columna in columnas_procesar:
        # Calcular el promedio de valores para cada categoría en cada periodo
        promedio_periodo1 = periodo1.groupby('Municipio')[columna].mean()
        promedio_periodo2 = periodo2.groupby('Municipio')[columna].mean()

        # Calcular el crecimiento porcentual promedio para cada categoría
        crecimiento_promedio = ((promedio_periodo2 - promedio_periodo1) / promedio_periodo1) * 100

        # Agregar el resultado al DataFrame resultados
        resultados = pd.concat([resultados, crecimiento_promedio], axis=1)

    # Renombrar las columnas con los nombres originales
    resultados.columns = columnas_procesar

    return resultados
print('calcular_crecimiento_promedio_por_categoria Loaded')

In [ ]:
def limitar_atipicos(df, columnas_procesar):
    # Inicializar un DataFrame para almacenar los resultados
    resultados = df.copy()

    # Iterar sobre las columnas a procesar
    for columna in columnas_procesar:
        # Calcular el rango intercuartílico (IQR)
        q1 = df[columna].quantile(0.25)
        q3 = df[columna].quantile(0.75)
        iqr = q3 - q1

        # Calcular los límites del bigote superior e inferior
        upper_limit = q3 + 1.5 * iqr
        lower_limit = q1 - 1.5 * iqr

        # Reemplazar los valores atípicos con los límites del bigote
        resultados[columna] = resultados[columna].apply(lambda x: upper_limit if x > upper_limit else lower_limit if x < lower_limit else x)

    return resultados
print('limitar_atipicos Loaded')

In [ ]:
def crear_dummies(df, columns_to_dummy):
    for col in columns_to_dummy:
        dummies = pd.get_dummies(df[col], prefix=col)
        df = pd.concat([df, dummies], axis=1)
        df.drop(col, axis=1, inplace=True)
    return df
print('crear_dummies Loaded')

In [ ]:
#Funcion para comvertir un data frame con años en columasn en filas y generar comparacion de fechas
def year_columnas(Industria, palabra,ID,Year_inicio_periodo1, Year_fin_periodo1, Year_inicio_periodo2, Year_fin_periodo2):

    new_columns = [col.replace(palabra, '') for col in Industria.columns] # quita un valor de una columna
    Industria.rename(columns=dict(zip(Industria.columns, new_columns)), inplace=True) # quita un valor de una columna

    Industria = Industria.melt(id_vars=[ID], var_name='Year', value_name='Valor') #Transpone los años en filas

    Industria.rename(columns={ID: 'Category'}, inplace=True) #Cambio de nombre de columna
    Industria=completar_datos_serie_tiempo(Industria,2008,2020) #Completa faltantes con interpolación

    Industria.rename(columns={'Category': 'Municipio'}, inplace=True) #Cambio de nombre de columna
    Industria.rename(columns={'Valor': 'Industria'}, inplace=True) #Cambio de nombre de columna
    lista_columnas = Industria[Industria.columns.difference(['Municipio','Year'])].columns.tolist()
    Industria=calcular_crecimiento_promedio_por_categoria(Industria,lista_columnas,Year_inicio_periodo1, Year_fin_periodo1, Year_inicio_periodo2, Year_fin_periodo2)
    Industria=limitar_atipicos(Industria, lista_columnas)
    Industria = Industria.fillna(0)
    Industria=Industria.reset_index()
    Industria.rename(columns={'index': 'Municipio'}, inplace=True) #Cambio de nombre de columna
    return Industria

In [ ]:
def train_and_evaluate_model(X_train, X_test, y_train, y_test, Modelo, param_grid=None, random_state=13):
    if Modelo not in ['xgboost', 'random_forest']:
        raise ValueError('Modelo debe ser "xgboost" o "random_forest".')

    if Modelo == 'xgboost':
        model = xgb.XGBClassifier(tree_method='gpu_hist', random_state=random_state)
        model = xgb.XGBClassifier(random_state=random_state)
    else:
        model = RandomForestClassifier(random_state=random_state)

    if param_grid is not None:
        grid_search = GridSearchCV(estimator=model, param_grid=param_grid, scoring='roc_auc_ovr', cv=5, n_jobs=-1)
        grid_search.fit(X_train, y_train)
        best_params = grid_search.best_params_
        model.set_params(**best_params)

    # Validación cruzada (k-fold cross-validation)
    cv_scores = cross_val_score(model, X_train, y_train, cv=5, scoring='roc_auc_ovr', n_jobs=-1)
    print("Mean CV ROC AUC:", cv_scores.mean())

    # Curva de aprendizaje
    #train_sizes, train_scores, test_scores = learning_curve(model, X_train, y_train, cv=5, scoring='roc_auc_ovr', n_jobs=-1)
    train_sizes, train_scores, test_scores = learning_curve(model, X_train, y_train, cv=5, scoring='roc_auc_ovr', n_jobs=-1)
    plt.figure()
    plt.plot(train_sizes, train_scores.mean(axis=1), 'o-', label='Training Score')
    plt.plot(train_sizes, test_scores.mean(axis=1), 'o-', label='Test Score')
    plt.xlabel('Training examples')
    plt.ylabel('ROC AUC')
    plt.legend(loc='best')
    plt.show()

    with tf.device('/GPU:0'):
        model.fit(X_train, y_train)

    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    roc_auc = roc_auc_score(y_test, model.predict_proba(X_test), multi_class='ovr')
    conf_matrix = confusion_matrix(y_test, y_pred)

    return model, accuracy, roc_auc, conf_matrix, best_params

In [ ]:
def evaluate_model(model, X_eval, y_eval):
    # Obtener las predicciones del modelo
    y_pred = model.predict(X_eval)

    # Calcular el accuracy
    accuracy = accuracy_score(y_eval, y_pred)

    # Calcular el valor de la curva ROC
    #roc_auc = roc_auc_score(y_eval, y_pred,multi_class='ovr')
    roc_auc = roc_auc_score(y_eval, model.predict_proba(X_eval), multi_class='ovr')
    # Calcular la matriz de confusión
    conf_matrix = confusion_matrix(y_eval, y_pred)

    # Crear el DataFrame con los resultados
    result_df = pd.DataFrame({
        'Accuracy': [accuracy],
        'ROC_AUC': [roc_auc]
    })

    return result_df,conf_matrix

In [ ]:
def obtener_clasificacion_y_scores(modelo, X, lista_id):
    """
    Obtiene la clasificación de y y el score de cada categoría para un conjunto de datos X.

    Parámetros:
        modelo: Modelo entrenado capaz de realizar predicciones multiclase.
        X: Conjunto de datos de entrada para realizar las predicciones.
        lista_id (list): Lista de identificadores (ID) correspondientes a los datos en X.

    Retorna:
        pandas DataFrame: Un DataFrame con las columnas ID, y, y las scores para cada categoría.
    """
    # Realizar predicciones para el conjunto de datos X
    y_pred_prob = modelo.predict_proba(X)

    # Obtener las categorías del modelo
    categorias = modelo.classes_

    # Crear una lista con el resultado para cada fila
    resultados = []
    for i in range(len(X)):
        fila = [lista_id[i]] + [categorias[np.argmax(y_pred_prob[i])]] + y_pred_prob[i].tolist()
        resultados.append(fila)

    # Crear el DataFrame con los resultados
    columnas = ['ID', 'y'] + [f'Score_{categoria}' for categoria in categorias]
    df_resultados = pd.DataFrame(resultados, columns=columnas)

    return df_resultados

In [ ]:
#Funcion que elimina atipicos superiores y los reeemplazo por el valor mayor
def identificar_y_reemplazar_atipicos_superiores(dataframe):
    # Copiar el DataFrame original para no modificarlo directamente
    df = dataframe.copy()

    # Iterar sobre cada columna del DataFrame
    for columna in df.columns:
        # Calcular los límites del boxplot para la columna actual
        q1 = df[columna].quantile(0.25)
        q3 = df[columna].quantile(0.75)
        iqr = q3 - q1
        limite_superior = q3 + 1.5 * iqr

        # Identificar los valores atípicos superiores en la columna actual
        valores_atipicos_superiores = df[columna] > limite_superior

        # Reemplazar los valores atípicos superiores por el límite superior del boxplot
        df.loc[valores_atipicos_superiores, columna] = limite_superior

    return df

In [ ]:
#Funcion que ajsuta una funcion especifica para completar atipicos
import pandas as pd
import numpy as np
from scipy.optimize import curve_fit

# Definir algunas funciones para probar
def funcion_lineal(x, a, b):
    return a * x + b

def funcion_exponencial(x, a, b):
    return a * np.exp(b * x)

def funcion_polinomica(x, a, b, c):
    return a * x**2 + b * x + c

def funcion_logaritmica(x, a, b):
    return a * np.log(x) + b

def funcion_coseno(x, a, b):
    return a * np.cos(b * x)

def funcion_seno(x, a, b):
    return a * np.sin(b * x)

def funcion_cuadratica(x, a, b, c):
    return a * x**2 + b * x + c

def funcion_cubica(x, a, b, c, d):
    return a * x**3 + b * x**2 + c * x + d

def funcion_raiz_cuadrada(x, a, b):
    return a * np.sqrt(x) + b

def funcion_logistica(x, a, b, c):
    return c / (1 + np.exp(-(x - b) / a))

import pandas as pd
import numpy as np
from scipy.optimize import curve_fit

def ajustar_funcion(df, columna_fecha, columna_valor, funciones, tolerancia_porcentaje):
    mejores_resultados = None
    mejor_mse = np.inf


    # Convertir las fechas a números enteros que representen el año
    df['Anio'] = df[columna_fecha].dt.year

    # Obtener los valores de año y valor como arrays
    anios = df['Anio'].values
    valores = df[columna_valor].values

    for funcion in funciones:
        # Ajustar la función a los datos de la serie de tiempo
        parametros_optimizados, _ = curve_fit(funcion, anios, valores,maxfev=20000)

        # Calcular los valores ajustados utilizando la función y los parámetros optimizados
        valores_ajustados = funcion(anios, *parametros_optimizados)

        # Calcular la diferencia entre los valores reales y los ajustados en porcentaje
        diferencia_porcentaje = 100 * np.abs((valores - valores_ajustados) / valores)

        # Reemplazar los valores que están fuera de la tolerancia con los valores ajustados
        valores_corregidos = np.where(diferencia_porcentaje > tolerancia_porcentaje, valores_ajustados, valores)

        # Calcular el error cuadrático medio entre los valores reales y los ajustados
        mse = np.mean((valores - valores_ajustados)**2)

        # Verificar si esta función es la de mejor ajuste hasta ahora
        if mse < mejor_mse:
            mejores_resultados = valores_corregidos
            mejor_mse = mse

    # Agregar una columna al DataFrame original con los valores ajustados de la mejor función
    df['Valores_Ajustados'] = mejores_resultados

    return df


def ajustar_funcion2(df, columna_fecha, columna_valor,columna_categoria, funciones, tolerancia_porcentaje ):
    df_resultado = df.copy()

    # Convertir las fechas a números enteros que representen el año
    df_resultado['Anio'] = df_resultado[columna_fecha].dt.year

    # Iterar por cada categoría única en la columna de categorías
    for categoria in df_resultado[columna_categoria].unique():
        df_categoria = df_resultado[df_resultado[columna_categoria] == categoria]

        # Obtener los valores de año y valor como arrays para la categoría actual
        anios_categoria = df_categoria['Anio'].values
        valores_categoria = df_categoria[columna_valor].values

        mejores_resultados = None
        mejor_mse = np.inf
        try:
          for funcion in funciones:
              #print(categoria)
              # Ajustar la función a los datos de la serie de tiempo de la categoría actual
              parametros_optimizados, _ = curve_fit(funcion, anios_categoria, valores_categoria, maxfev=100000)

              # Calcular los valores ajustados utilizando la función y los parámetros optimizados
              valores_ajustados = funcion(anios_categoria, *parametros_optimizados)

              # Calcular la diferencia entre los valores reales y los ajustados en porcentaje
              diferencia_porcentaje = 100 * np.abs((valores_categoria - valores_ajustados) / valores_categoria)

              # Reemplazar los valores que están fuera de la tolerancia con los valores ajustados
              valores_corregidos = np.where(diferencia_porcentaje > tolerancia_porcentaje, valores_ajustados, valores_categoria)

              # Calcular el error cuadrático medio entre los valores reales y los ajustados
              mse = np.mean((valores_categoria - valores_ajustados)**2)

              # Verificar si esta función es la de mejor ajuste hasta ahora
              if mse < mejor_mse:
                  mejores_resultados = valores_corregidos
                  mejor_mse = mse
        except Exception as e:
            # Capturar cualquier excepción que ocurra y continuar con el siguiente grupo
            print(f"Se produjo un error en el grupo '{categoria}': {str(e)}")
            continue
        # Agregar una columna al DataFrame de la categoría actual con los valores ajustados de la mejor función
        df_resultado.loc[df_resultado[columna_categoria] == categoria, 'Valores_Ajustados'] = mejores_resultados

    return df_resultado

In [ ]:
#Funcion que genera una rpediccion de arima
import pandas as pd
import numpy as np
from statsmodels.tsa.arima.model import ARIMA
from sklearn.metrics import mean_absolute_percentage_error

def generar_prediccion_arima(data, categoria, columna_fecha, columna_valor):
    # Filtrar los datos para la categoría y el rango de fechas deseados
    df_categoria = data[data['Categoria'] == categoria].copy()
    df_categoria[columna_fecha] = pd.to_datetime(df_categoria[columna_fecha])
    df_categoria.set_index(columna_fecha, inplace=True)

    # Dividir los datos en entrenamiento (2008-2017) y prueba (2018-2030)
    train_data = df_categoria['2008':'2017'][columna_valor]
    test_data = df_categoria['2018':'2030'][columna_valor]

    # Ajustar el modelo ARIMA para la serie de tiempo de entrenamiento
    modelo_arima = ARIMA(train_data, order=(1, 1, 0))
    resultado = modelo_arima.fit()

    # Generar el pronóstico para los próximos 13 años (2018-2030)
    pronostico_anual = resultado.forecast(steps=13)

    # Crear un rango de años para los próximos 13 años (2018-2030)
    fechas_pronostico = pd.date_range(df_categoria.index[-1], periods=13, freq='Y')

    # Crear un DataFrame con el pronóstico y los datos reales
    df_pronostico = pd.DataFrame({columna_fecha: fechas_pronostico, 'Pronostico': pronostico_anual})
    df_pronostico.set_index(columna_fecha, inplace=True)

    # Completar los datos reales del 2008 al 2017 en el DataFrame de pronóstico
    df_pronostico['Real'] = df_categoria['2008':'2017'][columna_valor]

    return df_pronostico

In [ ]:
#Quita atipicos por Boxplot y lso completa interpolando

def normalizar_serie_tiempo_por_categoria(df, fecha_col, categoria_col, valor_col):
    """
    Normaliza una serie de tiempo dentro de un DataFrame para cada categoría.

    Parámetros:
        - df: DataFrame con los datos.
        - fecha_col: Nombre de la columna que contiene las fechas.
        - categoria_col: Nombre de la columna que contiene las categorías.
        - valor_col: Nombre de la columna que contiene los valores de la serie de tiempo.

    Retorna:
        - DataFrame con la serie de tiempo normalizada por categoría.
    """

    def eliminar_outliers_interpolacion(serie_tiempo):
        q1 = serie_tiempo.quantile(0.25)
        q3 = serie_tiempo.quantile(0.75)
        iqr = q3 - q1
        limite_inferior = q1 - 1.5 * iqr
        limite_superior = q3 + 1.5 * iqr
        serie_tiempo_sin_outliers = serie_tiempo.mask((serie_tiempo < limite_inferior) | (serie_tiempo > limite_superior), other=None)
        serie_tiempo_completada = serie_tiempo_sin_outliers.interpolate(method='linear', limit_direction='both')
        return serie_tiempo_completada

    df_resultado = pd.DataFrame()
    #df=df.replace(0, np.nan, inplace=True)
    # Iterar por cada categoría única en el DataFrame
    for categoria in df[categoria_col].unique():
        # Filtrar los datos para la categoría actual
        df_categoria = df[df[categoria_col] == categoria].copy()

        # Eliminar valores atípicos y completar datos faltantes para la serie de tiempo de la categoría actual
        df_categoria[valor_col] = eliminar_outliers_interpolacion(df_categoria[valor_col])

        # Agregar los datos de la categoría actual al DataFrame resultado
        df_resultado = pd.concat([df_resultado, df_categoria])

    return df_resultado

In [ ]:
#Arima 10 años
import pandas as pd
import numpy as np
from statsmodels.tsa.arima.model import ARIMA

def ajustar_serie_tiempo_por_categoria(data, columna_categoria, columna_fecha, columna_valor, pronostico_anios=10):
    # Crear DataFrame vacío para almacenar los resultados
    df_resultado = pd.DataFrame()

    # Obtener lista de categorías únicas
    categorias_unicas = data[columna_categoria].unique()

    # Iterar sobre cada categoría
    for categoria in categorias_unicas:
        # Filtrar los datos para la categoría actual
        df_categoria = data[data[columna_categoria] == categoria].copy()

        # Convertir la columna de fecha a datetime (si no está en ese formato)
        df_categoria[columna_fecha] = pd.to_datetime(df_categoria[columna_fecha])

        # Establecer la columna de fecha como índice
        df_categoria.set_index(columna_fecha, inplace=True)

        # Ajustar el modelo ARIMA para la serie de tiempo actual
        modelo_arima = ARIMA(df_categoria[columna_valor], order=(1, 1, 0))
        resultado = modelo_arima.fit()

        # Generar el pronóstico para los próximos 10 años (anual)
        pronostico_anual = resultado.forecast(steps=pronostico_anios)

        # Crear un rango de fechas para los próximos 10 años (anual)
        fechas_pronostico = pd.date_range(df_categoria.index[-1], periods=pronostico_anios + 1, freq='Y')[1:]

        # Crear un DataFrame para el pronóstico y completar los NaN
        df_pronostico = pd.DataFrame({columna_fecha: fechas_pronostico, columna_valor: pronostico_anual})
        df_pronostico.set_index(columna_fecha, inplace=True)

        # Combinar el DataFrame de pronóstico con el DataFrame de la categoría actual
        df_categoria = df_categoria.append(df_pronostico)

        # Agregar la categoría al DataFrame de resultados
        df_categoria[columna_categoria] = categoria
        df_resultado = df_resultado.append(df_categoria)

    return df_resultado

In [ ]:
import pandas as pd
import numpy as np

def reemplazar_valores_negativos(df, categoria_col, valor_col):
    # Crear una copia del DataFrame para no modificar el original
    df_copia = df.copy()

    # Agrupar los datos por categoría
    grupos_categorias = df_copia.groupby(categoria_col)

    # Recorrer cada grupo de categoría
    for categoria, grupo in grupos_categorias:

        if grupo.empty:
            # Si el grupo está vacío, no hacer ninguna operación y continuar con el siguiente grupo
            continue
        # Obtener los valores de la serie de tiempo para la categoría actual
        valores = grupo[valor_col].values

         # Verificar si todos los valores en el grupo son cero
        if np.all(valores == 0):
            # Si todos son cero, no hacer ninguna operación y continuar con el siguiente grupo
            continue

        # Encontrar el menor número positivo presente en la serie
        minimo_positivo = np.min(valores[valores > 0])

        if minimo_positivo is np.nan:
            minimo_positivo = np.min(valores)
            minimo_positivo=minimo_positivo*-1

        # Reemplazar los valores negativos con el menor número positivo
        grupo.loc[grupo[valor_col] < 0, valor_col] = minimo_positivo

        # Actualizar el grupo modificado en el DataFrame copia
        df_copia.loc[grupo.index] = grupo

    return df_copia

In [ ]:
def comparacion_con_anio_base(df, categoria_col, valor_col, anio_col, anio_base):
    # Filtrar los datos para el año base
    datos_anio_base = df[df[anio_col] == anio_base].copy()

    # Renombrar la columna de valor para diferenciarla
    datos_anio_base.rename(columns={valor_col: f'{valor_col}_({anio_base})'}, inplace=True)

    # Merge para combinar los datos con el año base
    df_con_base = pd.merge(df, datos_anio_base[[categoria_col, f'{valor_col}_({anio_base})']],
                           on=categoria_col, how='left')

    # Calcular la comparación para cada año con respecto al año base
    for anio in df[anio_col].unique():
        if anio != anio_base:
            columna_comparacion = f'{valor_col}_({anio}-{anio_base})/{anio_base}'
            df_con_base[columna_comparacion] = (df_con_base[valor_col] -
                                                df_con_base[f'{valor_col}_({anio_base})']) / df_con_base[f'{valor_col}_({anio_base})']

    return df_con_base

In [ ]:
#Funcion que agrega los valores anterios de valorización respecto a la base
def agregar_valores_anteriores(df, categoria_col, valor_col, anios_anteriores=3):
    # Crear una copia del DataFrame para no modificar el original
    df_copia = df.copy()

    # Ordenar el DataFrame por categoría y año para asegurar que los valores estén en orden
    df_copia.sort_values([categoria_col, 'Year'], inplace=True)

    # Crear columnas nuevas para cada año anterior
    for i in range(1, anios_anteriores + 1):
        nombre_columna = f"{valor_col}_anterior_{i}"
        df_copia[nombre_columna] = df_copia.groupby(categoria_col)[valor_col].shift(i)

    return df_copia

In [ ]:
def agregar_valores_futuros(df, categoria_col, valor_col, anios_futuros=3):
    # Crear una copia del DataFrame para no modificar el original
    df_copia = df.copy()

    # Ordenar el DataFrame por categoría y año para asegurar que los valores estén en orden
    df_copia.sort_values([categoria_col, 'Year'], inplace=True)

    # Crear columnas nuevas para cada año futuro
    for i in range(1, anios_futuros + 1):
        nombre_columna = f"{valor_col}_futuro_{i}"
        df_copia[nombre_columna] = df_copia.groupby(categoria_col)[valor_col].shift(-i)

    return df_copia

In [ ]:
def columnas_con_20_por_ciento_ceros_o_nan(df):
    # Contar la cantidad de valores 0 o NaN en cada columna
    valores_ceros_o_nan = df.isin([0, np.nan]).sum()

    # Calcular el porcentaje de valores 0 o NaN en cada columna
    porcentaje_ceros_o_nan = valores_ceros_o_nan / len(df) * 100

    # Filtrar las columnas que tienen más del 60% de valores 0 o NaN
    columnas_con_mas_del_60_por_ciento = porcentaje_ceros_o_nan[porcentaje_ceros_o_nan > 20].index.tolist()

    return columnas_con_mas_del_60_por_ciento

In [ ]:
def normalizar_variable_entre_0_y_1(df, columna):
    # Obtener el valor mínimo y máximo de la columna
    min_valor = df[columna].min()
    max_valor = df[columna].max()

    # Aplicar la normalización min-max para que quede entre 0 y 1
    df[columna] = (df[columna] - min_valor) / (max_valor - min_valor)

    return df

In [ ]:
def normalize_variables_limpieza(dataset):
    # Eliminar valores infinitos o muy grandes
    dataset = dataset.replace([np.inf, -np.inf], np.nan)
    dataset = dataset.fillna(0)  # Reemplazar valores NaN por cero o cualquier otro valor apropiado

    scaler = MinMaxScaler()
    normalized_dataset = scaler.fit_transform(dataset)
    normalized_dataset = pd.DataFrame(normalized_dataset, columns=dataset.columns)
    return normalized_dataset

In [ ]:
def eliminar_outliers_iqr(df, variable_col):
    Q1 = df[variable_col].quantile(0.25)
    Q3 = df[variable_col].quantile(0.75)
    IQR = Q3 - Q1

    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR

    df_sin_outliers = df[(df[variable_col] >= lower_bound) & (df[variable_col] <= upper_bound)]
    return df_sin_outliers

In [ ]:
def eliminar_registros_con_cero(df, columna):
    # Filtrar las filas que contienen cero en la columna especificada
    df_sin_cero = df[df[columna] != 0]

    return df_sin_cero

In [ ]:

from scipy.stats import ttest_ind

def efecto_por_categoria(df, columna_categoria, columna_numerica, poblacion, nivel_confianza=0.10):
    """
    Determina si la diferencia de promedios para cada categoría es significativa en relación con la población,
    la dirección del efecto y la magnitud del efecto.

    Parámetros:
    - df: DataFrame que contiene la columna de categoría y la columna numérica a analizar.
    - columna_categoria: Nombre de la columna que contiene las categorías.
    - columna_numerica: Nombre de la columna que contiene el valor numérico a analizar.
    - poblacion: DataFrame que representa la población completa con la columna 'columna_numerica'.
    - nivel_confianza: Nivel de confianza para realizar la prueba t (por defecto es 0.95).

    Retorna:
    - DataFrame que muestra el resultado del análisis, indicando si la diferencia de promedios
      para cada categoría es significativa en relación con la población, la dirección del efecto
      y la magnitud del efecto.
    """
    resultados = []

    media_poblacion = poblacion[columna_numerica].mean()
    #print(media_poblacion)
    magnitudes = {
        'Muy Bajo': 5,
        'Bajo': 10,
        'Medio': 15,
        'Alto': 20,
        'Muy Alto':30
    }



    for categoria in df[columna_categoria].unique():
        data_por_categoria = df[df[columna_categoria] == categoria][columna_numerica]
        t_stat, p_value = ttest_ind(data_por_categoria, poblacion[columna_numerica])
        #print(categoria)
        #print(data_por_categoria.mean())
        #print('p_value')
        #print(p_value)
        #print('t_stat')
        #print(t_stat)
        #print(data_por_categoria.mean() - media_poblacion)
        #print((data_por_categoria.mean() - media_poblacion)*100/media_poblacion)

        if p_value < (1 - nivel_confianza):
            diferencia_promedios = data_por_categoria.mean() - media_poblacion

            if diferencia_promedios > 0:
                direccion = "Positivo"
            elif diferencia_promedios < 0:
                direccion = "Negativo"
            else:
                direccion = "Indeterminado"

            Porcentaje_diferencia=abs(diferencia_promedios/media_poblacion)*100

            for mag, limite in magnitudes.items():
                if Porcentaje_diferencia <= limite:
                    magnitud = mag
                    break
            else:
                magnitud = "Indeterminado"

            resultado = f"{direccion} - {magnitud}"
        else:
            resultado = "No significativo"

        resultados.append(resultado)

    resultados_df = pd.DataFrame(resultados, index=df[columna_categoria].unique(), columns=[columna_numerica])

    return resultados_df